<a href="https://colab.research.google.com/github/zqiaohe/100daysofcode/blob/master/CupIT2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Расчёт CLTV

In [0]:
#датасет
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/CupIT2020/train_data_last.csv",index_col=[1], parse_dates=[1] ) 
#df = pd.read_csv("/content/drive/My Drive/CupIT2020/train_data.csv") 

##Полный датасет

In [0]:
%%time
#датасет 
df1 = df.sort_index(ascending=False)
#оконная функция, собирающая сумму за полгода 
win_sum = df1.groupby(['cif_id']).gi_smooth_3m.rolling(window=6).sum()
#оконная функция, собирающая сумму за 2 месяца 
win_sum2 = df1.groupby(['cif_id']).gi_smooth_3m.rolling(window=2).sum()
#оконная функция, собирающая сумму за 3 месяца 
win_sum3 = df1.groupby(['cif_id']).gi_smooth_3m.rolling(window=3).sum()
#оконная функция, собирающая gi_smooth_3m следующего месяца
next_month = df1.groupby(['cif_id']).gi_smooth_3m.apply(lambda x: x.rolling(2, 1).sum().shift().bfill())

CPU times: user 4min 2s, sys: 2.12 s, total: 4min 4s
Wall time: 4min 4s


In [0]:
%%time
#оконная функция, собирающая gi_smooth_3m следующего месяца
df1 = df.sort_index(ascending=False)
next_month = df1.groupby(['cif_id']).gi_smooth_3m.apply(lambda x: x.rolling(2, 1).sum().shift().bfill())

In [0]:
%%time
#реиндексируем и джоиним
win_sum = win_sum.to_frame()
win_sum.reset_index(inplace=True)
win_sum2 = win_sum2.to_frame()
win_sum2.reset_index(inplace=True)
win_sum3 = win_sum3.to_frame()
win_sum3.reset_index(inplace=True)
next_month = next_month.to_frame()
next_month.reset_index(inplace=True)

In [0]:
%%time
train = df1.merge(win_sum, on = ['cif_id', 'dlk_cob_date'], suffixes=('', '_pred6'))
train

,dlk_cob_date,cif_id,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,cur_quantity_mort,cur_quantity_cc,cur_quantity_deposits,cur_quantity_dc,cur_quantity_accounts,cur_quantity_saccounts,cur_quantity_mf,cc_balance,cl_balance,ml_balance,pl_balance,td_volume,ca_volume,sa_volume,mf_volume,dc_cash_spend_v,dc_cash_spend_c,cc_cash_spend_v,cc_cash_spend_c,dc_pos_spend_v,dc_pos_spend_c,cc_pos_spend_v,cc_pos_spend_c,ca_f,rc_session_qnt_cur_mon,cur_qnt_sms,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level,gi_smooth_3m_pred6
0,2018-12-31,23504,8.363642,OTH,0,03. Высшее,ЗДРАВООХРАНЕНИЕ,QUALIFIED STAFF,1,0,0,1,0,1,1,0,0,2.39040,NaN,NaN,NaN,NaN,30.50184,NaN,NaN,79.2,1,0.0,0,46.29216,10,0.00000,0,1,18,1.0,1,0,0,0,597.675360,34.0,82,92.0,1,3,NaN
1,2018-12-31,502568,41.809906,MLN,1,03. Высшее,"Финансы, управляющие компании, банки",QUALIFIED STAFF,0,1,0,4,0,2,2,0,0,15.51672,NaN,NaN,-8694.63072,NaN,3628.57320,NaN,NaN,1104.0,3,225.6,1,641.09328,25,130.15200,4,1,18,2.0,1,0,0,0,1560.000000,32.0,44,7.0,1,3,NaN
2,2018-12-31,449111,103.665309,MSK,0,03. Высшее,ДРУГОЕ,MIDDLE MANAGEMENT,1,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-56002.92168,NaN,1212.88824,NaN,NaN,1920.0,2,0.0,0,1797.03888,20,0.00000,0,1,3,1.0,1,0,0,0,3513.762960,45.0,57,20.0,1,3,NaN
3,2018-12-31,496588,84.323978,MLN,1,03. Высшее,ТОРГОВЛЯ,QUALIFIED STAFF,1,1,0,2,0,1,2,0,0,0.00000,NaN,NaN,-10387.46232,NaN,36.14664,NaN,NaN,492.0,3,0.0,0,338.85864,30,0.00000,0,1,26,1.0,1,0,0,0,791.820000,31.0,95,78.0,1,3,NaN
4,2018-12-31,510933,96.825091,OTH,1,03. Высшее,Государственная служба,QUALIFIED STAFF,0,0,0,2,0,1,1,0,0,-5048.12232,NaN,NaN,NaN,NaN,0.00024,NaN,NaN,0.0,0,0.0,0,0.00000,0,458.26656,15,1,8,-100.0,1,1,0,1,1056.348346,35.0,106,101.0,1,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4161778,2018-01-31,449802,3.266337,OTH,1,01. Среднее,"ПРОИЗВОДСТВО, ДОБЫВАЮЩАЯ ПРОМЫШЛЕННОСТЬ",03,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-183.10224,NaN,17.56656,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.00000,0,1,0,-100.0,1,1,0,1,720.000000,48.0,133,64.0,1,1,6.399792
4161779,2018-01-31,122158,2.608516,OTH,1,03. Высшее,"ПРОИЗВОДСТВО, ДОБЫВАЮЩАЯ ПРОМЫШЛЕННОСТЬ",BLUE COLLAR STAFF,1,1,0,2,0,1,1,0,0,0.00000,NaN,NaN,-1200.39408,NaN,262.35576,NaN,NaN,1440.0,3,0.0,0,29.90040,4,0.00000,0,1,14,1.0,1,0,0,0,771.124800,31.0,13,88.0,1,3,209.911235
4161780,2018-01-31,362383,5.025090,MLN,1,NaN,NaN,NaN,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,52.94568,NaN,NaN,708.0,4,0.0,0,1167.61680,56,0.00000,0,1,6,1.0,1,0,0,0,1190.992560,44.0,19,118.0,0,-100,29.243668
4161781,2018-01-31,395634,3.162025,MSK,0,03. Высшее,ДРУГОЕ,TRADE REPRESENTATIVE,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,314.60064,NaN,NaN,708.0,1,0.0,0,155.95992,10,0.00000,0,1,0,1.0,1,0,0,0,1557.432000,38.0,53,55.0,1,3,31.778090


In [0]:
win_sum2

cif_id  dlk_cob_date
1001    2018-12-31             NaN
        2018-11-30       21.908271
        2018-10-31      144.683255
        2018-09-30      220.462897
        2018-08-31      183.229231
                           ...    
511537  2018-05-31       85.800848
        2018-04-30       85.386530
        2018-03-31       85.377042
        2018-02-28       87.307184
        2018-01-31      114.659481
Name: gi_smooth_3m, Length: 4161783, dtype: float64

In [0]:
%%time
train2 = train.merge(win_sum2, on = ['cif_id', 'dlk_cob_date'], suffixes=('', '_pred2'))
train

CPU times: user 2.76 s, sys: 788 ms, total: 3.54 s
Wall time: 3.56 s


In [0]:
%%time
train3 = train2.merge(win_sum3, on = ['cif_id', 'dlk_cob_date'], suffixes=('', '_pred3'))
train3

CPU times: user 2.9 s, sys: 9.26 s, total: 12.2 s
Wall time: 12.2 s


In [0]:
%%time
train3.to_csv(r'/content/drive/My Drive/CupIT2020/train_data_last.csv', index = False, header=True)

CPU times: user 3min 21s, sys: 2.03 s, total: 3min 23s
Wall time: 3min 29s


In [0]:
train3

,dlk_cob_date,cif_id,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,cur_quantity_mort,cur_quantity_cc,cur_quantity_deposits,cur_quantity_dc,cur_quantity_accounts,cur_quantity_saccounts,cur_quantity_mf,cc_balance,cl_balance,ml_balance,pl_balance,td_volume,ca_volume,sa_volume,mf_volume,dc_cash_spend_v,dc_cash_spend_c,cc_cash_spend_v,cc_cash_spend_c,dc_pos_spend_v,dc_pos_spend_c,cc_pos_spend_v,cc_pos_spend_c,ca_f,rc_session_qnt_cur_mon,cur_qnt_sms,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level,gi_smooth_3m_pred6,gi_smooth_3m_pred2,gi_smooth_3m_pred3
0,2018-12-31,23504,8.363642,OTH,0,03. Высшее,ЗДРАВООХРАНЕНИЕ,QUALIFIED STAFF,1,0,0,1,0,1,1,0,0,2.39040,NaN,NaN,NaN,NaN,30.50184,NaN,NaN,79.2,1,0.0,0,46.29216,10,0.00000,0,1,18,1.0,1,0,0,0,597.675360,34.0,82,92.0,1,3,NaN,NaN,NaN
1,2018-12-31,502568,41.809906,MLN,1,03. Высшее,"Финансы, управляющие компании, банки",QUALIFIED STAFF,0,1,0,4,0,2,2,0,0,15.51672,NaN,NaN,-8694.63072,NaN,3628.57320,NaN,NaN,1104.0,3,225.6,1,641.09328,25,130.15200,4,1,18,2.0,1,0,0,0,1560.000000,32.0,44,7.0,1,3,NaN,NaN,NaN
2,2018-12-31,449111,103.665309,MSK,0,03. Высшее,ДРУГОЕ,MIDDLE MANAGEMENT,1,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-56002.92168,NaN,1212.88824,NaN,NaN,1920.0,2,0.0,0,1797.03888,20,0.00000,0,1,3,1.0,1,0,0,0,3513.762960,45.0,57,20.0,1,3,NaN,NaN,NaN
3,2018-12-31,496588,84.323978,MLN,1,03. Высшее,ТОРГОВЛЯ,QUALIFIED STAFF,1,1,0,2,0,1,2,0,0,0.00000,NaN,NaN,-10387.46232,NaN,36.14664,NaN,NaN,492.0,3,0.0,0,338.85864,30,0.00000,0,1,26,1.0,1,0,0,0,791.820000,31.0,95,78.0,1,3,NaN,NaN,NaN
4,2018-12-31,510933,96.825091,OTH,1,03. Высшее,Государственная служба,QUALIFIED STAFF,0,0,0,2,0,1,1,0,0,-5048.12232,NaN,NaN,NaN,NaN,0.00024,NaN,NaN,0.0,0,0.0,0,0.00000,0,458.26656,15,1,8,-100.0,1,1,0,1,1056.348346,35.0,106,101.0,1,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4161778,2018-01-31,449802,3.266337,OTH,1,01. Среднее,"ПРОИЗВОДСТВО, ДОБЫВАЮЩАЯ ПРОМЫШЛЕННОСТЬ",03,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-183.10224,NaN,17.56656,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.00000,0,1,0,-100.0,1,1,0,1,720.000000,48.0,133,64.0,1,1,6.399792,5.303105,6.151317
4161779,2018-01-31,122158,2.608516,OTH,1,03. Высшее,"ПРОИЗВОДСТВО, ДОБЫВАЮЩАЯ ПРОМЫШЛЕННОСТЬ",BLUE COLLAR STAFF,1,1,0,2,0,1,1,0,0,0.00000,NaN,NaN,-1200.39408,NaN,262.35576,NaN,NaN,1440.0,3,0.0,0,29.90040,4,0.00000,0,1,14,1.0,1,0,0,0,771.124800,31.0,13,88.0,1,3,209.911235,10.814775,27.786994
4161780,2018-01-31,362383,5.025090,MLN,1,NaN,NaN,NaN,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,52.94568,NaN,NaN,708.0,4,0.0,0,1167.61680,56,0.00000,0,1,6,1.0,1,0,0,0,1190.992560,44.0,19,118.0,0,-100,29.243668,9.855897,14.025579
4161781,2018-01-31,395634,3.162025,MSK,0,03. Высшее,ДРУГОЕ,TRADE REPRESENTATIVE,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,314.60064,NaN,NaN,708.0,1,0.0,0,155.95992,10,0.00000,0,1,0,1.0,1,0,0,0,1557.432000,38.0,53,55.0,1,3,31.778090,9.296621,16.392250


#Чистим данные 

In [0]:
df['cc_balance'] = df['cc_balance'].fillna(0)
df['cl_balance'] = df['cl_balance'].fillna(0)
df['ml_balance'] = df['ml_balance'].fillna(0)
df['pl_balance'] = df['pl_balance'].fillna(0)
df['td_volume'] = df['td_volume'].fillna(0)
df['ca_volume'] = df['ca_volume'].fillna(0)
df['sa_volume'] = df['sa_volume'].fillna(0)
df['mf_volume'] = df['mf_volume'].fillna(0)
df['cu_education_level'] = df['cu_education_level'].fillna('unknown')
df['cu_empl_area'] = df['cu_empl_area'].fillna('unknown')
df['cu_empl_level'] = df['cu_empl_level'].fillna('unknown')

In [3]:
df["big_city"].replace("MSK", 3, inplace = True)
df["big_city"].replace("OTH", 2, inplace = True)
df["big_city"].replace("MLN", 1, inplace = True)
df["big_city"].replace("SPB", 0, inplace = True)
df["cu_education_level"].replace("unknown", 4, inplace = True)
df["cu_education_level"].replace("03. Высшее", 3, inplace = True)
df["cu_education_level"].replace("02. Среднее профессиональное", 2, inplace = True)
df["cu_education_level"].replace("01. Среднее", 1, inplace = True)
df["cu_education_level"].replace("04. Второе высшее, ученая степень", 0, inplace = True)
df.head()

,dlk_cob_date,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,cur_quantity_mort,cur_quantity_cc,cur_quantity_deposits,cur_quantity_dc,cur_quantity_accounts,cur_quantity_saccounts,cur_quantity_mf,cc_balance,cl_balance,ml_balance,pl_balance,td_volume,ca_volume,sa_volume,mf_volume,dc_cash_spend_v,dc_cash_spend_c,cc_cash_spend_v,cc_cash_spend_c,dc_pos_spend_v,dc_pos_spend_c,cc_pos_spend_v,cc_pos_spend_c,ca_f,rc_session_qnt_cur_mon,cur_qnt_sms,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level,gi_smooth_3m_pred6,gi_smooth_3m_pred2,gi_smooth_3m_pred3
cif_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23504,2018-12-31,8.363642,2,0,3,ЗДРАВООХРАНЕНИЕ,QUALIFIED STAFF,1,0,0,1,0,1,1,0,0,2.39040,0.0,0.0,0.00000,0.0,30.50184,0.0,0.0,79.2,1,0.0,0,46.29216,10,0.00000,0,1,18,1.0,1,0,0,0,597.675360,34.0,82,92.0,1,3,NaN,NaN,NaN
502568,2018-12-31,41.809906,1,1,3,"Финансы, управляющие компании, банки",QUALIFIED STAFF,0,1,0,4,0,2,2,0,0,15.51672,0.0,0.0,-8694.63072,0.0,3628.57320,0.0,0.0,1104.0,3,225.6,1,641.09328,25,130.15200,4,1,18,2.0,1,0,0,0,1560.000000,32.0,44,7.0,1,3,NaN,NaN,NaN
449111,2018-12-31,103.665309,3,0,3,ДРУГОЕ,MIDDLE MANAGEMENT,1,1,0,0,0,1,2,0,0,0.00000,0.0,0.0,-56002.92168,0.0,1212.88824,0.0,0.0,1920.0,2,0.0,0,1797.03888,20,0.00000,0,1,3,1.0,1,0,0,0,3513.762960,45.0,57,20.0,1,3,NaN,NaN,NaN
496588,2018-12-31,84.323978,1,1,3,ТОРГОВЛЯ,QUALIFIED STAFF,1,1,0,2,0,1,2,0,0,0.00000,0.0,0.0,-10387.46232,0.0,36.14664,0.0,0.0,492.0,3,0.0,0,338.85864,30,0.00000,0,1,26,1.0,1,0,0,0,791.820000,31.0,95,78.0,1,3,NaN,NaN,NaN
510933,2018-12-31,96.825091,2,1,3,Государственная служба,QUALIFIED STAFF,0,0,0,2,0,1,1,0,0,-5048.12232,0.0,0.0,0.00000,0.0,0.00024,0.0,0.0,0.0,0,0.0,0,0.00000,0,458.26656,15,1,8,-100.0,1,1,0,1,1056.348346,35.0,106,101.0,1,3,NaN,NaN,NaN


In [0]:
#дропаем нули
train = df.dropna(subset=['gi_smooth_3m_pred6'])

In [0]:
train.to_csv(r'/content/drive/My Drive/CupIT2020/train_witnout_nulls_6.csv', index = False, header=True)

#Отбираем признаки

In [0]:
import numpy as np

In [0]:
train = pd.read_csv("/content/drive/My Drive/CupIT2020/train_witnout_nulls_6.csv") 

In [0]:
target = train.gi_smooth_3m_pred6
data =  train.drop(['gi_smooth_3m_pred6', 'gi_smooth_3m_pred2', 'gi_smooth_3m_pred3'], axis=1)

In [0]:
from sklearn import preprocessing

In [0]:
#import pandas as pd
#from sklearn.preprocessing import LabelBinarizer

jobs_encoder1 = LabelBinarizer()
jobs_encoder1.fit(data['dlk_cob_date'])
transformed = jobs_encoder1.transform(data['dlk_cob_date'])
data['dlk_cob_date'] = pd.DataFrame(transformed)

In [35]:
%%time
jobs_encoder2 = LabelBinarizer()
jobs_encoder2.fit(data['cu_empl_area'])
transformed = jobs_encoder2.transform(data['cu_empl_area'])
data['cu_empl_area'] = pd.DataFrame(transformed)

CPU times: user 21 s, sys: 5.5 s, total: 26.5 s
Wall time: 26.5 s


In [36]:
%%time
jobs_encoder3 = LabelBinarizer()
jobs_encoder3.fit(data['cu_empl_level'])
transformed = jobs_encoder3.transform(data['cu_empl_level'])
data['cu_empl_level'] = pd.DataFrame(transformed)

CPU times: user 13.5 s, sys: 82 ms, total: 13.6 s
Wall time: 13.6 s


In [0]:
from sklearn import preprocessing

In [0]:
scaled = preprocessing.scale(data)

In [40]:
scaled

array([[-0.40794443, -0.10469682,  0.03417311, ..., -0.90962251,
        -1.33235764,  0.27857613],
       [-0.40794443, -0.75872893, -1.9558091 , ...,  0.33231808,
         0.75054923,  0.31354033],
       [-0.40794443, -0.77603472,  1.02916422, ...,  1.00105533,
        -1.33235764,  0.31354033],
       ...,
       [ 2.45131426, -0.68391447, -0.96081799, ...,  0.29137499,
        -1.33235764, -3.28777166],
       [ 2.45131426, -0.72086978,  1.02916422, ..., -0.56843004,
         0.75054923,  0.31354033],
       [ 2.45131426, -0.3417014 , -0.96081799, ..., -0.78679322,
         0.75054923,  0.31354033]])

In [0]:
#data =  data.drop(['cu_empl_area'], axis=1)
#data =  data.drop(['cu_empl_level'], axis=1)
#data =  data.drop(['dlk_cob_date'], axis=1)

In [43]:
# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression
# create a base classifier used to evaluate a subset of attributes
model = LinearRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 20)
rfe = rfe.fit(scaled,target)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True False False False False False  True False  True  True False
 False False False False  True  True  True  True  True  True False False
  True False  True  True  True  True False  True False False False False
  True  True  True False False False False False False]
[ 1  1 15 19 21 26 25  1 10  1  1 13  9 14  8 23  1  1  1  1  1  1  2 12
  1  7  1  1  1  1 20  1 22  6  3 11  1  1  1  4 24 18  5 16 17]


#MAPE

In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [0]:
y_true = np.array([3, -0.5, 2, 7])
y_pred = np.array([2.5, -0.3, 2, 8])

In [0]:
mean_absolute_percentage_error(y_true, y_pred)

17.738095238095237

In [0]:
test_df = pd.read_csv("/content/drive/My Drive/CupIT2020/test_data.csv") 

#Мусор

##Пробуем на мелком датасете

In [0]:
%%time
#датасет из трёх индексов
mini_df = df[(df['cif_id']==335992)|(df['cif_id']==234815)|(df['cif_id']==198842)].sort_index(ascending=False)
#оконная функция, собирающая сумму за полгода по отсортированному в обратном порядке индексу
win_sum = mini_df.groupby(['cif_id']).gi_smooth_3m.rolling(window=6).sum()

CPU times: user 42.4 ms, sys: 3.1 ms, total: 45.5 ms
Wall time: 49 ms


In [0]:
#реиндексируем и джоиним
win_sum = win_sum.to_frame()
win_sum.reset_index(inplace=True)
mini_df.reset_index(inplace=True)

In [0]:
mergedDf = mini_df.merge(win_sum, on = ['cif_id', 'dlk_cob_date'], suffixes=('', '_pred6'))
mergedDf